In [25]:
!pip install geopy

In [6]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 7.6 MB/s 
     |████████████████████████████████| 6.3 MB 43.0 MB/s 
     |████████████████████████████████| 16.7 MB 459 kB/s 


In [97]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
#import plotly_express as px
import geopandas as gpd
import geopy as gp
from geopy.geocoders import GoogleV3

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

In [98]:
df = pd.read_csv('/content/drive/MyDrive/reduced_data.csv')

In [61]:
df

,Captured Time,Latitude,Longitude,Value,Unit,Height
0,2022-03-19 01:59:32,37.637260,139.660360,0.0,cpm,60.0
1,2022-03-19 01:59:31,37.637260,139.660360,37.0,cpm,60.0
2,2022-03-19 01:59:20,42.565000,-70.783000,13.0,cpm,10.0
3,2022-03-19 01:59:19,42.565000,-70.783000,24.0,cpm,10.0
4,2022-03-19 01:59:19.611,37.442836,-122.128094,29.0,cpm,3.0
...,...,...,...,...,...,...
37693290,2014-01-20 20:42:09,40.001931,-105.248908,31.0,cpm,1.0
37693291,2013-05-26 10:36:32,60.202004,24.921344,54.0,cpm,44.0
37693292,NaN,35.745945,139.918206,21.0,cpm,1.0
37693293,2012-12-02 05:59:30,37.508102,139.932585,33.0,cpm,1.0


In [99]:
df.drop_duplicates(subset=['Latitude', 'Longitude', 'Value'], inplace = True)

In [9]:
df = df[df.Value != 0]

In [12]:
df

,Captured Time,Latitude,Longitude,Value,Unit,Height
0,2022-03-19 01:59:31,37.637260,139.660360,37.0,cpm,60.0
1,2022-03-19 01:59:20,42.565000,-70.783000,13.0,cpm,10.0
2,2022-03-19 01:59:19,42.565000,-70.783000,24.0,cpm,10.0
3,2022-03-19 01:59:19.611,37.442836,-122.128094,29.0,cpm,3.0
4,2022-03-19 01:59:19.602,37.442836,-122.128094,12.0,cpm,3.0
...,...,...,...,...,...,...
493705,2014-01-21 17:25:37,40.001931,-105.248908,31.0,cpm,1.0
493706,2013-05-26 10:36:32,60.202004,24.921344,54.0,cpm,44.0
493707,NaN,35.745945,139.918206,21.0,cpm,1.0
493708,2012-12-02 05:59:30,37.508102,139.932585,33.0,cpm,1.0


In [100]:
df.sort_values(by = ['Latitude', 'Longitude'], inplace = True)

In [101]:
df = df.reset_index(drop=True)

In [32]:
df

,Captured Time,Latitude,Longitude,Value,Unit,Height
0,2017-09-21 22:21:04,-90.000000,37.442836,10.0,cpm,5.0
1,2019-06-07 21:27:19,-47.392879,127.472214,14.0,cpm,173.0
2,2019-06-07 21:12:19,-47.392879,127.472214,13.0,cpm,173.0
3,2019-06-07 21:27:19,-47.392879,127.472214,41.0,cpm,173.0
4,2019-06-07 21:12:19,-47.392879,127.472214,38.0,cpm,173.0
...,...,...,...,...,...,...
493705,2017-08-02 12:55:33,90.000000,140.471962,8.0,cpm,60.0
493706,2017-08-01 19:04:39,90.000000,140.471962,21.0,cpm,60.0
493707,2017-07-28 09:38:11,90.000000,140.471962,60.0,cpm,60.0
493708,2017-07-28 03:02:25,90.000000,140.471962,36.0,cpm,60.0


In [102]:
import math
import statistics

data = []

Latitude, Longitude, Value, Height = 0, 0, 0, 0

Latitude_tab, Longitude_tab, Value_tab, Height_tab = [[] for i in range(4)]
#print(type(Latitude_tab))
for index, row in df.iterrows():
    if Latitude == 0 and Longitude == 0 and Value == 0 and Height == 0:
      Latitude, Longitude, Value, Height = row['Latitude'], row['Longitude'], row['Value'], row['Height']
      #print(type(Latitude_tab))
      Latitude_tab.append(row['Latitude'])
      Longitude_tab.append(row['Longitude'])
      Value_tab.append(row['Value'])
      Height_tab.append(row['Height'])
    else:
      distance = math.sqrt((Latitude - row['Latitude'])**2 + (Longitude- row['Longitude'])**2) 
      if distance < 0.2:  #wtedy są blisko i można potraktować je z pewnością jako ten sam kraj (czyli przyłączamy wartości średnie do)
        Latitude_tab.append(row['Latitude'])
        Longitude_tab.append(row['Longitude'])
        Value_tab.append(row['Value'])
        Height_tab.append(row['Height'])
      else:
          data.append([statistics.mean(Latitude_tab), statistics.mean(Longitude_tab), statistics.mean(Value_tab), statistics.mean(Height_tab)])
          Latitude_tab, Longitude_tab, Value_tab, Height_tab = [[] for i in range(4)]
          Latitude, Longitude, Value, Height = 0, 0, 0, 0
cdf = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'Value', 'Height'])



In [103]:
cdf

,Latitude,Longitude,Value,Height
0,-90.000000,37.442836,10.000000,5.000000
1,-47.392879,127.472214,29.600000,173.000000
2,-36.852148,174.763045,32.862069,150.681992
3,0.000000,-102.000000,32.073171,96.000000
4,0.000000,0.096181,30.500000,96.928571
...,...,...,...,...
180,50.954568,7.107926,13.500000,69.000000
181,52.200865,4.985461,26.725599,-6.200737
182,52.291160,4.544818,25.375000,-6.750000
183,52.392100,4.877244,25.898039,19.511765


In [104]:
def get_country(row):
    locator = Nominatim(user_agent="ekspllo3", timeout=10)
    pos = str(row['Latitude']) + ', ' + str(row['Longitude'])
    location = locator.reverse(pos, timeout = 10, language="en")
    try:
      return location.raw['address']['country']
    except:
      return "no data" 

In [105]:
cdf['country'] = cdf.apply(lambda row: get_country(row), axis = 1)

In [106]:
cdf.sort_values(by = "country")

,Latitude,Longitude,Value,Height,country
166,43.391711,-79.824522,153.333333,2.000000,Canada
7,22.285870,114.146747,80.224610,149.171577,China
178,49.030717,8.347995,31.348837,115.000000,Germany
180,50.954568,7.107926,13.500000,69.000000,Germany
179,49.864983,8.627907,8.250000,100.000000,Germany
...,...,...,...,...,...
4,0.000000,0.096181,30.500000,96.928571,no data
3,0.000000,-102.000000,32.073171,96.000000,no data
1,-47.392879,127.472214,29.600000,173.000000,no data
5,1.000000,2.000000,15.400000,3.000000,no data


In [107]:
cdf['country'].value_counts()

United States    96
Japan            57
South Korea      13
no data           6
Germany           3
Netherlands       3
New Zealand       1
Somalia           1
China             1
Spain             1
Canada            1
Italy             1
Slovenia          1
Name: country, dtype: int64

In [84]:
print(get_country(df.iloc[888]))

Somalia
